<a href="https://colab.research.google.com/github/sije91/DRD./blob/master/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

import csv
import pandas as pd 
import numpy as np  
import re  
import nltk  
from sklearn.datasets import load_files  
nltk.download('stopwords')  
import pickle  
from nltk.corpus import stopwords
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score


from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, numpy, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
from keras.preprocessing import text

import gensim 
from gensim.models import Word2Vec

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Convert labels from string to integer (the numbers are equal to the value number in the gold standard for example 'disgust': 1)

In [0]:
df = pd.read_csv("/content/DRC_topics.csv", delimiter = ';')

df = df[df['event.message'].notnull()]
def get_length(message):
    """
    """
    message_size = len(message)
    return int(message_size)

convert_string = {'anger': 4, 'disgust': 1, 'fear': 7, 'happiness': 6,'neutral': 3,'sadness': 5,'surprise': 2}

def get_integer_emotion(emotion):
    """
    """
    new_integer = convert_string[emotion] 
    return int(new_integer)

df['size_message'] = df['event.message'].apply(get_length)
df['emotion_integer'] = df['event.emotion'].apply(get_integer_emotion)
df = df[df['size_message'] > 3]

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Get equal number of labels for each class.

In [0]:
df1 = df['emotion_integer'] == 1
df11 = df[df1][0:3700]

df2 = df['emotion_integer'] == 2
df22 = df[df2][0:3700]

df3 = df['emotion_integer'] == 3
df33 = df[df3][0:3700]

df4 = df['emotion_integer'] == 4
df44 = df[df4][0:3700]

df5 = df['emotion_integer'] == 5
df55 = df[df5][0:3700]

df6 = df['emotion_integer'] == 6
df66 = df[df6][0:3700]

df7 = df['emotion_integer'] == 7
df77 = df[df7][0:3700]

frames = [df11, df22, df33, df44, df55, df66, df77]
df = pd.concat(frames)


Stemming method for French data

In [0]:
documents = []

from nltk.stem.snowball import FrenchStemmer

stemmer = FrenchStemmer()
def get_stemming(sen):  
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(sen))

    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 

    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)

    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)

    # Converting to Lowercase
    document = document.lower()

    # Lemmatization
    document = document.split()

    document = [stemmer.stem(word) for word in document]
    document = ' '.join(document)

    return(document)

In [0]:
df_emo = pd.read_csv("/content/French_goldstandard.csv", delimiter = ',', error_bad_lines=False)

df_emo["word_french_stem"] = df_emo["word_french"].apply(get_stemming)

df_emo.to_csv('French_goldstandard2.csv')

In [0]:
df['message_stemmer'] = df['event.message'].apply(get_stemming)

Calculate the score for each class based on the goldstandard, the class with the most high score will be used as label.

In [0]:
import csv
df = df[0:200000]
def get_score(word_list):
    split_words = word_list.split(' ')
    for word in split_words:
        #print(word)
        with open('French_goldstandard2.csv') as csvfile:
            csv_values = csv.reader(csvfile)
            for row in csv_values:
                disgust = 0
                surprise = 0
                neutral = 0
                anger = 0
                sad = 0
                happy = 0
                fear = 0 
                if str(row[11]) == str(word):
                    disgust += float(row[3])
                    surprise += float(row[4])
                    neutral += float(row[5])
                    anger += float(row[6])
                    sad += float(row[7])
                    happy += float(row[8])
                    try:
                        fear += float(row[9])  
                    except ValueError:
                        fear += 0
                    scores = [disgust,surprise,neutral,anger,sad,happy,fear]
                    scores.append(scores.index(max(scores)))
                    return(scores)

df['message_score'] = df['message_stemmer'].apply(get_score)
df.to_csv('DRC_topics_scores.csv',sep = ';')

In [0]:
df_scores = pd.read_csv("/home/sije/Thesis/DRC_topics_scores.csv", delimiter = ';')

df_scores = df_scores[df_scores.message_score.notnull()]
print(len(df_scores))
def get_class(score_list):
    scores = score_list.split(',')
    #print(scores)
    return(int(scores[6].replace(']','')) + 1)

df_scores['message_class'] = df_scores['message_score'].apply(get_class)    
print(df_scores['message_class'].value_counts())

In [0]:
df1 = df_scores.loc[df_scores['message_class'] == 1]['emotion_integer'][0:1500]

df2 = df_scores.loc[df_scores['message_class'] == 2]['emotion_integer'][0:1500]

df3 = df_scores.loc[df_scores['message_class'] == 3]['emotion_integer'][0:1500]

df4 = df_scores.loc[df_scores['message_class'] == 4]['emotion_integer'][0:1500]

df5 = df_scores.loc[df_scores['message_class'] == 5]['emotion_integer'][0:1500]

df6 = df_scores.loc[df_scores['message_class'] == 6]['emotion_integer'][0:1500]

df7 = df_scores.loc[df_scores['message_class'] == 7]['emotion_integer'][0:1500]

frames = [df1, df2, df3, df4, df5, df6, df7]
df = pd.concat(frames)
print(len(df))

In [0]:
df = df_scores.dropna()
X_train, X_test, y_train, y_test = model_selection.train_test_split(df_scores['message_stemmer'], df_scores['message_class'])

In [0]:
# label encode the target variable 
encoder = preprocessing.LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)

In [0]:
# create a count vectorizer object 
#count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect = CountVectorizer()
count_vect.fit(df['message_stemmer'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(X_train.apply(lambda x: np.str_(x)))
xvalid_count =  count_vect.transform(X_test.apply(lambda x: np.str_(x)))

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(df['message_stemmer'])
xtrain_tfidf =  tfidf_vect.transform(X_train.apply(lambda x: np.str_(x)))
xvalid_tfidf =  tfidf_vect.transform(X_test.apply(lambda x: np.str_(x)))

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(df['message_stemmer'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(X_train.apply(lambda x: np.str_(x)))
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(X_test.apply(lambda x: np.str_(x)))

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(df['message_stemmer'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(X_train.apply(lambda x: np.str_(x)))
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(X_test.apply(lambda x: np.str_(x)))

In [0]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    print(confusion_matrix(y_test,predictions))  
    print(classification_report(y_test,predictions))  
    print(accuracy_score(y_test,predictions))
    #return metrics.accuracy_score(predictions, y_test)

In [0]:
# Naive Bayes on Count Vectors
print("NB, Count Vectors: ")
train_model(naive_bayes.MultinomialNB(), xtrain_count, y_train, xvalid_count)


# Naive Bayes on Word Level TF IDF Vectors
print("NB, WordLevel TF-IDF: ")
train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, y_train, xvalid_tfidf)

# Naive Bayes on Ngram Level TF IDF Vectors
print("NB, N-Gram Vectors: ")
train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, y_train, xvalid_tfidf_ngram)


# Naive Bayes on Character Level TF IDF Vectors
print("NB, CharLevel Vectors: ")
train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, y_train, xvalid_tfidf_ngram_chars)

In [0]:
# Linear Classifier on Count Vectors
print("LR, Count Vectors: ")
train_model(linear_model.LogisticRegression(), xtrain_count, y_train, xvalid_count)


# Linear Classifier on Word Level TF IDF Vectors
print("LR, WordLevel TF-IDF: ")
train_model(linear_model.LogisticRegression(), xtrain_tfidf, y_train, xvalid_tfidf)


# Linear Classifier on Ngram Level TF IDF Vectors
print("LR, N-Gram Vectors: ")
train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, y_train, xvalid_tfidf_ngram)


# Linear Classifier on Character Level TF IDF Vectors
print("LR, CharLevel Vectors: ")
train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, y_train, xvalid_tfidf_ngram_chars)
